In [1]:
from pynq import Overlay, MMIO, Interrupt
import asyncio
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import random
import csv
from rig.type_casts import float_to_fp, fp_to_float, NumpyFloatToFixConverter, NumpyFixToFloatConverter

In [2]:
ol = Overlay("system_wrapper.bit")
ol?

In [3]:
# IP Definition
action_ram_0 = ol.RAM_BlockRAM_0axi_bram_ctrl_0 
action_ram_1 = ol.RAM_BlockRAM_1axi_bram_ctrl_0
action_ram_2 = ol.RAM_BlockRAM_2axi_bram_ctrl_0
action_ram_3 = ol.RAM_BlockRAM_3axi_bram_ctrl_0
register_mem = ol.intelight_mem_0
#interrupt_ctrl = ol.PS_axi_intc_0          

In [4]:
signed_15_16 = float_to_fp(signed=True, n_bits=32, n_frac=16)
unsigned_8_8 = float_to_fp(signed=False, n_bits=16, n_frac=8)
unsigned_0_3 = float_to_fp(signed=False, n_bits=3, n_frac=3)
toFloat_3f = fp_to_float(n_frac=3)
toFloat_32f = fp_to_float(n_frac=16)

def twos_comp(val_in, bits):
    val = int(val_in, 16)
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val                         # return positive value as is

# Control Register Inputs

In [5]:
# Setting alpha
alpha = 0.5
f_alpha = unsigned_0_3(alpha)
alpha = toFloat_3f(f_alpha)
print("Alpha = {:.3f} (0b{:04b})".format(alpha, f_alpha))

#Setting Gamma
gamma = 0.5
f_gamma = unsigned_0_3(gamma)
gamma = toFloat_3f(f_gamma)
print("Gamma = {:.3f} (0b{:04b})".format(gamma, f_gamma))

#Setting delta_t
delta_t = 0.2
f_delta_t = unsigned_0_3(delta_t)
delta_t = toFloat_3f(f_delta_t)
print("Delta t = {:.3f} (0b{:04b})".format(delta_t, f_delta_t))

Alpha = 0.500 (0b0100)
Gamma = 0.500 (0b0100)
Delta t = 0.125 (0b0001)


In [6]:
#Setting LSFR Seed
# random.seed(13217053)
# seed_float = random.random()
seed_float = 10
seed_fixedp = unsigned_8_8(seed_float)
print("Seed = {:f} 0b{:016b} 0x{:04x}".format(seed_float, seed_fixedp, seed_fixedp))

Seed = 10.000000 0b0000101000000000 0x0a00


In [7]:
# Setting MaxStep, MaxGeneration
max_step = 100
max_episode = 5800
print("Max Step = {:d} 0b{:016b} 0x{:04x}".format(max_step, max_step, max_step))
print("Max Episode = {:d} 0b{:016b} 0x{:04x}".format(max_episode, max_episode, max_episode))

Max Step = 100 0b0000000001100100 0x0064
Max Episode = 5800 0b0001011010101000 0x16a8


In [8]:
#Setting Reward Value
reward_0 = 0x00000000
reward_1 = 0x00140000
reward_2 = 0x00460000

print("Reward 0 = 0x{:08X}".format(reward_0))
print("Reward 1 = 0x{:08X}".format(reward_1))
print("Reward 2 = 0x{:08X}".format(reward_2))

Reward 0 = 0x00000000
Reward 1 = 0x00140000
Reward 2 = 0x00460000


In [9]:
# Setting debit
debit_out = 0x001F_0000
debit_0 = 0x0005_0000
debit_1 = 0x000A_0000
debit_2 = 0x000F_0000
debit_3 = 0x0014_0000
print("Debit out = 0x{:08X}".format(debit_out))
print("Debit 0 = 0x{:08X}".format(debit_0))
print("Debit 1 = 0x{:08X}".format(debit_1))
print("Debit 2 = 0x{:08X}".format(debit_2))
print("Debit 3 = 0x{:08X}".format(debit_3))

Debit out = 0x001F0000
Debit 0 = 0x00050000
Debit 1 = 0x000A0000
Debit 2 = 0x000F0000
Debit 3 = 0x00140000


In [10]:
# Setting initial congestion length
init_traffic_0 = 0x0015_0000
init_traffic_1 = 0x0015_0000
init_traffic_2 = 0x0015_0000
init_traffic_3 = 0x0015_0000
print("Init taffic 0 = 0x{:08X}".format(init_traffic_0))
print("Init taffic 1 = 0x{:08X}".format(init_traffic_1))
print("Init taffic 2 = 0x{:08X}".format(init_traffic_2))
print("Init taffic 3 = 0x{:08X}".format(init_traffic_3))

Init taffic 0 = 0x00150000
Init taffic 1 = 0x00150000
Init taffic 2 = 0x00150000
Init taffic 3 = 0x00150000


In [11]:
# Setting level border
limit_level_0 = 0x000A_0000
limit_level_1 = 0x0014_0000
limit_level_2 = 0x001E_0000
print("Level 0 border = 0x{:08X}".format(limit_level_0))
print("Level 1 border = 0x{:08X}".format(limit_level_1))
print("Level 2 border = 0x{:08X}".format(limit_level_2))

Level 0 border = 0x000A0000
Level 1 border = 0x00140000
Level 2 border = 0x001E0000


In [12]:
# Input setting to register memory
reg = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
reg[0] |= f_alpha<<23
reg[0] |= f_gamma<<20
reg[0] |= f_delta_t<<17
# reg[0] |= start<<16
reg[0] |= seed_fixedp
reg[1] |= max_step<<16
reg[1] |= max_episode
reg[2] |= debit_out
reg[3] |= debit_0
reg[4] |= debit_1
reg[5] |= debit_2
reg[6] |= debit_3
reg[7] |= init_traffic_0
reg[8] |= init_traffic_1
reg[9] |= init_traffic_2
reg[10]|= init_traffic_3
reg[11]|= limit_level_0
reg[12]|= limit_level_1
reg[13]|= limit_level_2
reg[14]|= reward_0
reg[15]|= reward_1
reg[16]|= reward_2

for i in range(17):
    register_mem.write(i*4,reg[i])
    print("0b{:032b}(0x{:08X})".format(reg[i], reg[i]))

0b00000010010000100000101000000000(0x02420A00)
0b00000000011001000001011010101000(0x006416A8)
0b00000000000111110000000000000000(0x001F0000)
0b00000000000001010000000000000000(0x00050000)
0b00000000000010100000000000000000(0x000A0000)
0b00000000000011110000000000000000(0x000F0000)
0b00000000000101000000000000000000(0x00140000)
0b00000000000101010000000000000000(0x00150000)
0b00000000000101010000000000000000(0x00150000)
0b00000000000101010000000000000000(0x00150000)
0b00000000000101010000000000000000(0x00150000)
0b00000000000010100000000000000000(0x000A0000)
0b00000000000101000000000000000000(0x00140000)
0b00000000000111100000000000000000(0x001E0000)
0b00000000000000000000000000000000(0x00000000)
0b00000000000101000000000000000000(0x00140000)
0b00000000010001100000000000000000(0x00460000)


In [13]:
for i in range (17):
    temp = register_mem.read(i*4)
    print("{:32X}".format(temp))

                         2420A00
                          6416A8
                          1F0000
                           50000
                           A0000
                           F0000
                          140000
                          150000
                          150000
                          150000
                          150000
                           A0000
                          140000
                          1E0000
                               0
                          140000
                          460000


# Hardware Learning

In [15]:
async def wait_for_interrupt(interrupt):
    register_mem.write(0,reg[0]|0x00010000)
    register_mem.write(0,reg[0]|0x00000000)
    start = time.time()
    print("Waiting for interrupt...")
    while True:
        await interrupt.wait()
        end = time.time()
        total_time = end-start
        print("Done")
        print("Execution time = {0}".format(total_time))
        break

In [16]:
ol.interrupt_pins

{'PS/axi_intc_0/intr': {'controller': 'PS/axi_intc_0',
  'index': 0,
  'fullpath': 'PS/axi_intc_0/intr'},
 'finish': {'controller': 'PS/axi_intc_0', 'index': 0, 'fullpath': 'finish'},
 'CU_0/finish': {'controller': 'PS/axi_intc_0',
  'index': 0,
  'fullpath': 'CU_0/finish'}}

In [17]:
loop = asyncio.get_event_loop()
intr_inst  = Interrupt("PS/axi_intc_0/intr")
task = loop.create_task(wait_for_interrupt(intr_inst))
loop.run_until_complete(task)

Waiting for interrupt...
Done
Execution time = 0.004399776458740234


In [18]:
num_of_road = 4
num_of_segment = 4
NUM_OF_STATE = 4*4*4*4
NUM_OF_ACTION = 4
bufferTable = np.zeros((NUM_OF_STATE, NUM_OF_ACTION))

for i in range(0, NUM_OF_STATE, 1):
    row_mem = np.zeros(0)
    row_mem = np.append(row_mem, twos_comp(hex(action_ram_0.read(i*4)), 32))
    row_mem = np.append(row_mem, twos_comp(hex(action_ram_1.read(i*4)), 32))
    row_mem = np.append(row_mem, twos_comp(hex(action_ram_2.read(i*4)), 32))
    row_mem = np.append(row_mem, twos_comp(hex(action_ram_3.read(i*4)), 32))
    bufferTable[i] = row_mem

frac_bit = 16
fp_to_float = NumpyFixToFloatConverter(frac_bit)
qTable = fp_to_float(bufferTable)

In [19]:
#Print Hardware Q-Table from learning
print("========== Hardware Q-Table ==========")
print(qTable)

========== Hardware Q-Table ==========
[[  0.           0.           0.           0.        ]
 [  0.           0.           0.           0.        ]
 [  0.           0.           0.           0.        ]
 ...
 [ 68.41362    135.30838013 136.7467804  138.27999878]
 [ 69.99882507 139.99990845 139.99990845 139.85151672]
 [  0.           0.           0.           0.        ]]


In [20]:
# Write to CSV
header = ['North', 'East', 'West', 'South']
with open('Q_Matrix_Traffic.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(qTable)